In [1]:
#Libraries 
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import subprocess
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev
import stl
from stl import mesh
import numpy as np
import os
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from PIL import Image, ImageTk
import trimesh
import trimesh.viewer
import trimesh.transformations as tf

#
class App(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        #self.create_widgets()
        self.master.title("TurbineGenerator UNB-FURNAS")
        self.master.configure(bg="#ffffff")   

      
        photo = []      
        
        image = Image.open("turbineGUIheaderRound.png")
        # resize image
        new_size = (469, 171)  # set new size here
        image = image.resize(new_size)   
        photo = ImageTk.PhotoImage(image)
        # Create a Label widget to display the image
        header = tk.Label(self.master, image=photo)
        header.image = photo  # Keep a reference to the PhotoImage to prevent it from being garbage collected
        header.grid(row=0, column=0, columnspan=3, rowspan=5)
        
       
       
        self.towerRadius= tk.IntVar(value=1)
        self.towerHeight= tk.IntVar(value=15)
        self.output = tk.StringVar(value="torre.stl")
         
       

    
   
        
        tk.Label(self.master, text="Tower radius:",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=1, column=3)
        tk.Entry(self.master, textvariable=self.towerRadius, width=10).grid(row=1, column=4)
        
        tk.Label(self.master, text="Tower height:",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=3, column=3)
        tk.Entry(self.master, textvariable=self.towerHeight, width=10).grid(row=3, column=4)
        
        tk.Label(self.master, text="Output File",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=2, column=3)
        tk.Entry(self.master, textvariable=self.output).grid(row=2, column=4)
        
 
        tk.Button(self.master, text="Generate Tower!", command=self.run_script, width=50,
                  fg="black", bg="green",font=("Helvetica", 12, "bold")).grid(row=4, column=3,columnspan=2)
        
        
    def browse_file1(self):
        self.file1_path.set(filedialog.askopenfilename())
    def browse_file2(self):
        self.file2_path.set(filedialog.askopenfilename())
               


        
    def run_script(self):
        
        towerHeight=self.towerHeight.get()
        towerRadius=self.towerRadius.get()
        
        

        # Define circle parameters
        r = towerRadius
        npoints = 100


        theta = np.linspace(0, 2*np.pi, npoints)

        # Define circle points
        circle1 = np.zeros((npoints, 3))
        circle1[:, 0] = r*np.cos(theta)
        circle1[:, 1] = r*np.sin(theta)+10
        circle1[:, 2] = 0

        circle2 = np.zeros((npoints, 3))
        circle2[:, 0] = r*np.cos(theta)
        circle2[:, 1] = r*np.sin(theta)+10
        circle2[:, 2] = -towerHeight

        # Concatenate circles and reshape
        vertices = np.concatenate([circle1, circle2]).reshape(-1, 3)

        # Create faces
        faces = []
        for i in range(npoints-1):
            faces.append([i, i+1, i+npoints+1])
            faces.append([i, i+npoints+1, i+npoints])

        # Connect last point to first point
        faces.append([npoints-1, 0, npoints])
        faces.append([npoints-1, npoints, 2*npoints-1])

        # Save solid in STL format
        solid = stl.mesh.Mesh(np.zeros(len(faces), dtype=stl.mesh.Mesh.dtype))
        for k, f in enumerate(faces):
            for m in range(3):
                solid.vectors[k][m] = vertices[f[m]]

        solid.save("tower_C.stl")

 


        filenames=[]
        filenames.append("tower_C.stl")
        #filenames.append('sphere.stl')
        #filenames.append('circle_b.stl')
        #filenames.append('last_profile.stl')


        combined_mesh = stl.mesh.Mesh(np.zeros(0, dtype=stl.mesh.Mesh.dtype))

        # Loop over filenames
        for filename in filenames:
            # Load current mesh
            mesh = stl.mesh.Mesh.from_file(filename)

            # Combine current mesh with previous meshes
            combined_mesh = stl.mesh.Mesh(np.concatenate([combined_mesh.data, mesh.data]))

        # Save combined mesh to file
        outputFilename=self.output.get()
        combined_mesh.save(outputFilename)



        os.remove("tower_C.stl")
     
        
        
        self.fig = Figure(figsize=(4, 3), dpi=100)
        self.canvas = FigureCanvasTkAgg(self.fig, self.master)
        self.canvas.get_tk_widget().grid(row=5, column=3, columnspan=2, rowspan=2)
        
        
        mesh = trimesh.load(outputFilename)
        self.fig.clear()
        ax = self.fig.add_subplot(111, projection='3d')
        ax.plot_trisurf(mesh.vertices[:,0], mesh.vertices[:,1], mesh.vertices[:,2], triangles=mesh.faces)#, edgecolor="k",linewidth=0.01)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        min_y = np.min(mesh.vertices[:, 1])
        max_y = np.max(mesh.vertices[:, 1])
        # Remove blank space
        tight_layout = {'pad': 0.1}
        self.fig.tight_layout(rect=[0, 0, 1, 1], **tight_layout)
        #step_size = max(3, (max_y - min_y) // 10)  # choose step size of at least 3, or 1/10 of range if that's larger
       # y_values = np.arange(min_y, max_y, step_size)
        ax.yaxis.set_ticks([min_y, max_y])

        ax.set_zlabel('Z')
        ax.set_aspect('equal')
        self.canvas.draw()
        





root = tk.Tk()
app = App(master=root)
app.grid()
app.mainloop()
